In [1]:
!pip install kaggle

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data collection - Kaggle API**

In [3]:
kaggle_dict = json.load(open('kaggle.json'))

In [4]:
#kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dict["username"]
os.environ["KAGGLE_KEY"] = kaggle_dict["key"]

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
#unzip the dataset file

with ZipFile("imdb-dataset-of-50k-movie-reviews.zip",'r') as zip_ref:
  zip_ref.extractall()

In [7]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading the dataset & data preprocess**

In [8]:
data = pd.read_csv('IMDB Dataset.csv')

In [9]:
data.shape

(50000, 2)

In [10]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [11]:
data.replace({"sentiment" : {"positive" : 1,"negative" : 0}},inplace = True)

In [12]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
#splitting training data and test data

train_data , test_data = train_test_split(data,test_size=0.2,random_state=42)

In [14]:
tokenizer = Tokenizer(num_words = 5000)

tokenizer.fit_on_texts(train_data['review'])

In [18]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [19]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [20]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [21]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [22]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [23]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


**Building LSTM** :
mainly used for sequential data

In [25]:
#build the model

model = Sequential()
model.add(Embedding(input_dim = 5000,output_dim = 128,input_length = 200))
model.add(LSTM(128,dropout = 0.2,recurrent_dropout = 0.2))
model.add(Dense(1,activation='sigmoid'))

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

**Training the model**

In [29]:
model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 202s 405ms/step - loss: 0.2462 - accuracy: 0.9026 - val_loss: 0.2899 - val_accuracy: 0.8792
Epoch 2/5
500/500 [==============================] - 199s 399ms/step - loss: 0.2011 - accuracy: 0.9218 - val_loss: 0.3480 - val_accuracy: 0.8716
Epoch 3/5
500/500 [==============================] - 195s 391ms/step - loss: 0.1926 - accuracy: 0.9265 - val_loss: 0.3424 - val_accuracy: 0.8704
Epoch 4/5
500/500 [==============================] - 195s 390ms/step - loss: 0.1559 - accuracy: 0.9413 - val_loss: 0.3476 - val_accuracy: 0.8773
Epoch 5/5
500/500 [==============================] - 203s 406ms/step - loss: 0.1356 - accuracy: 0.9498 - val_loss: 0.4353 - val_accuracy: 0.8777


**Model Evaluation**

In [30]:
loss , accuracy = model.evaluate(X_test,Y_test)

313/313 [==============================] - 22s 70ms/step - loss: 0.4264 - accuracy: 0.8789


In [31]:
print(f"test loss : {loss}")
print(f"test accuracy : {accuracy}")

test loss : 0.4264325797557831
test accuracy : 0.8788999915122986


**Build Predictive System**

In [40]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence,maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
  return sentiment

In [45]:
new_review = 'good'
sentiment = predict_sentiment(new_review)
print(sentiment)

1/1 [==============================] - 0s 107ms/step
positive
